In [15]:
#Dependency
from Bio import Entrez
import pytaxonkit as ptax
import pandas as pd
import urllib

In [16]:
#Input
tax_ids=set(pd.read_csv(snakemake.input[0],sep="\t")["TaxID"])

species_names=ptax.name(tax_ids,data_dir=os.path.dirname(snakemake.input[-1]))["Name"]
#Preparations
os.makedirs(snakemake.output[0], exist_ok=True)
os.makedirs(snakemake.output[1], exist_ok=True)
#TODO read from config
Entrez.email = snakemake.params[0]
#Lookup
for index,i in enumerate(tax_ids):
    term="txid"+str(i)+" [Organism:exp] AND representative [PROP] AND latest [Filter]'"
    handle=Entrez.esearch(db="assembly", retmax=10, term=term)
    records=Entrez.read(handle)
    handle.close()
    try:
        best_match=records["IdList"][0]
    except:
        term="txid"+str(i)+" [Organism:exp]"
        handle=Entrez.esearch(db="assembly", retmax=10, term=term)
        records=Entrez.read(handle)
        handle.close()
        best_match=records["IdList"][0]
        
    handle=Entrez.esummary(db="assembly",id=best_match)
    record=Entrez.read(handle)
    handle.close()
    #TODO put safety net in place
    #assert(species_name.upper().strip() in record[0]['Title'].upper().strip())\n",
    #handle=Entrez.efetch(db="assembly",id=best_match,rettype="fasta",retmode="text")
    #out_fasta=handle.read()
    #handle.close()
    url=record['DocumentSummarySet']['DocumentSummary'][0]['FtpPath_RefSeq']
    if not url:
        url=record['DocumentSummarySet']['DocumentSummary'][0]['FtpPath_GenBank']
    #Output
    outfile=snakemake.output[0]+"/"+str(i)+"/genome.fa.gz"
    os.makedirs(os.path.dirname(outfile), exist_ok=True)
    #with open(outfile, "w",newline="") as f:
    #    print(out_fasta,file=f)
    urllib.request.urlretrieve(url+"/"+os.path.basename(url)+"_genomic.fna.gz", outfile)
    urllib.request.urlretrieve(url+"/"+os.path.basename(url)+"_genomic.gbff.gz", snakemake.output[1]+"/"+str(i)+".gbff.gz")
    #handle=Entrez.efetch(db="assembly",id=best_match,rettype="gbwithparts",retmode="text")
    #out_gbff=handle.read()
    #handle.close()
    
    #Output
    #outfile=snakemake.output[1]+"/"+str(i)+".gbff"
    #with open(outfile, "w",newline="") as f:
    #    print(out_gbff,file=f)
    
#https://dmnfarrell.github.io/bioinformatics/assemblies-genbank-python